# Emulate v8

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load_hdf
from kepler.pandas.decorators import RingerDecorator
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy
import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

## Decorator:

In [3]:
def generator( df ):
    columns= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[columns].values.astype(np.float32)
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    rings = norm1(rings)
    return [rings]

In [4]:
tpath = 'output/fitting'

In [5]:
decorators = [
    RingerDecorator('ringer_v8_tight' , tpath + '/ElectronRingerTightTriggerConfig.conf'    , generator),
    RingerDecorator('ringer_v8_medium', tpath + '/ElectronRingerMediumTriggerConfig.conf'   , generator),
    RingerDecorator('ringer_v8_loose' , tpath + '/ElectronRingerLooseTriggerConfig.conf'    , generator),
    RingerDecorator('ringer_v8_vloose', tpath + '/ElectronRingerVeryLooseTriggerConfig.conf', generator),
]

2022-03-13 13:21:20,228 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerTightTriggerConfig.conf
2022-03-13 13:21:20,821 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerMediumTriggerConfig.conf
2022-03-13 13:21:21,300 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerLooseTriggerConfig.conf
2022-03-13 13:21:21,784 | Py.RingerDecorator                      INFO Reading... output/fitting/ElectronRingerVeryLooseTriggerConfig.conf


2022-03-13 13:21:20.259907: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-03-13 13:21:20.259936: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2022-03-13 13:21:20.259952: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba51): /proc/driver/nvidia/version does not exist
2022-03-13 13:21:20.260158: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-13 13:21:20.292514: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-03-13 13:21:20.301021: I tensorflow/compiler/xla/service/service.c

In [6]:
def load_in_loop( paths, decorators ):

    tables = []
    for path in progressbar( paths, prefix='Reading files...'):
        df = load_hdf( path )
        hold_these_columns = []
        for decorator in decorators:
            decorator.apply(df) 
            hold_these_columns.append(decorator.column)
            hold_these_columns.append(decorator.column + '_output')
            
        tables.append(df[hold_these_columns])

    return pd.concat(tables).reset_index(drop=True)

## Read all 2017 bins:

In [7]:
dpath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
dpath+= '/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = []
for et in range(5):
    for eta in range(5):
        paths.append( dpath.format(ET=et,ETA=eta) )

In [8]:
my_df = load_in_loop( paths, decorators )

Reading files...: 100%|██████████| 25/25 [16:35<00:00, 39.81s/it]


In [9]:
my_df.to_hdf(output_path+'/data17_table_v8.h5', key='df', mode='w')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->boolean,key->block1_values] [items->Index(['ringer_v8_tight', 'ringer_v8_medium', 'ringer_v8_loose',
       'ringer_v8_vloose'],
      dtype='object')]

  encoding=encoding,


## Read all 2018 bins:

In [10]:
dpath = '/home/jodafons/public/cern_data/new_files/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
dpath+= '/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = []
for et in range(5):
    for eta in range(5):
        paths.append( dpath.format(ET=et,ETA=eta) )

In [11]:
my_df = load_in_loop( paths, decorators )

Reading files...: 100%|██████████| 25/25 [19:36<00:00, 47.05s/it]


In [12]:
my_df.to_hdf(output_path+'/data18_table_v8.h5', key='df', mode='w')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->boolean,key->block1_values] [items->Index(['ringer_v8_tight', 'ringer_v8_medium', 'ringer_v8_loose',
       'ringer_v8_vloose'],
      dtype='object')]

  encoding=encoding,


## Read all mc16 boosted bins:

In [13]:
dpath = '/home/jodafons/public/cern_data/new_files/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2'
dpath+= '/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2_et{ET}_eta{ETA}.h5'
paths = []
for et in range(5):
    for eta in range(5):
        paths.append( dpath.format(ET=et,ETA=eta) )

In [14]:
my_df = load_in_loop( paths, decorators )

Reading files...: 100%|██████████| 25/25 [00:06<00:00,  3.93it/s]


In [15]:
my_df.to_hdf(output_path+'/mc16_table_v8.h5', key='df', mode='w')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->boolean,key->block1_values] [items->Index(['ringer_v8_tight', 'ringer_v8_medium', 'ringer_v8_loose',
       'ringer_v8_vloose'],
      dtype='object')]

  encoding=encoding,
